In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib.animation import FuncAnimation, HTMLWriter
from IPython.core.display import HTML

# Wykorzystane wzory:

Funkcja falowa: $\psi(x,t) = \psi_{\text{re}}(x,t) + i\psi_{\text{im}}(x,t)$

$\psi_{\text{re}}(x , t + \Delta t) = \psi_{\text{re}}(x, t) - \frac{\hbar\Delta t}{2  m_\text{e}(\Delta x)^2}\left[ \psi_{\text{im}}\left(x+\Delta x, t + \frac{\Delta t}{2}\right) - 2\psi_{\text{im}}\left(x, t + \frac{\Delta t}{2}\right) + \psi_{\text{im}}\left(x-\Delta x, t + \frac{\Delta t}{2}\right) \right] + \frac{\Delta t}{\hbar}V(x)\psi_{\text{im}}\left(x, t + \frac{\Delta t}{2}\right)$
$\psi_{\text{im}}(x , t + \frac{3}{2}\Delta t) = \psi_{\text{im}}\left(x, t + \frac{\Delta t}{2}\right) + \frac{\hbar\Delta t}{2  m_\text{e}(\Delta x)^2}\left[ \psi_{\text{re}}\left(x+\Delta x, t + \Delta t\right) - 2\psi_{\text{re}}\left(x, t + \Delta t\right) + \psi_{\text{re}}\left(x-\Delta x, t + \Delta t\right) \right] - \frac{\Delta t}{\hbar}V(x)\psi_{\text{re}}\left(x, t + \Delta t\right)$

Wartość oczekiwana energii: $\langle\text{E}\rangle = \langle\text{KE}\rangle + \langle\text{PE}\rangle$

$\langle\text{KE}\rangle = \frac{\hbar^2}{2m(\Delta x)^2}\sum\limits_{n = 1}^{N}\psi^{*}_{n}\cdot\left[ \psi_{n+1} - 2\psi_{n} + \psi_{n-1} \right]\cdot(x_{n} - x_{n-1})$

$\langle\text{PE}\rangle = \sum\limits_{n = 1}^{N}\psi^{*}_{n}\cdot V(x_n)\cdot\psi_{n}\cdot(x_{n} - x_{n-1})$

Prawdopodobieństwo: $P(x,t) = \left[\psi_{\text{re}}(x,t)\right]^2 + \psi_{\text{im}}(x, t + \frac{\Delta t}{2})\cdot\psi_{\text{im}}(x, t - \frac{\Delta t}{2}) $

$$\Psi(x, 0) = \sqrt[4]{\frac{2a}{\pi}}\cdot \text{e}^{-ax^2}$$

In [2]:
dt = 0.005 * 10**(-15)
dx = 0.1 * 10**(-9)
hbar = 1.054 * 10**(-34)
m = 0.91 * 10**(-30)

In [3]:
def gauss(x, a):
    return (2 * a / np.pi)**(1/4) * np.exp(-a * x**2)

# Cząstka swobodna: $V(x) = 0$

In [28]:
class Psi:
    ra = (hbar/(2*m)) * (dt/(dx**2))
    a = 1.0
    Re = []
    Im = []
    Im_r = []
    Im_f = []
    P = []
    X = []
    V = []
    def initialise(safe):
        x = -15
        while(x <= 15):
            Psi.Re.append(gauss(x, Psi.a))
            Psi.Im.append(0.0)
            Psi.Im_r.append(0.0)
            Psi.Im_f.append(0.0)
            Psi.V.append(0.0) 
            Psi.P.append(gauss(x, Psi.a)**2)
            Psi.X.append(x)
            x += 0.1 
    def update(safe, n):
        Re = Psi.Re
        Im = Psi.Im
        for k in range(0, n):
            for i in range(1, len(Psi.Re) - 1):
                Psi.Re[i] = Re[i] - Psi.ra * (Im[i + 1] - 2 * Im[i] + Im[i - 1]) + dt/hbar * Psi.V[i] * Im[i]
                Psi.Im[i] = Im[i] + Psi.ra * (Re[i + 1] - 2 * Re[i] + Re[i - 1]) - dt/hbar * Psi.V[i] * Re[i]               
                Psi.Im_r[i] = Im[i]
                Psi.Im_f[i] = Psi.Im[i] + Psi.ra * (Psi.Re[i + 1] - 2 * Psi.Re[i] + Psi.Re[i - 1]) - dt/hbar * Psi.V[i] * Re[i] 
                Psi.P[i] = Psi.Re[i]**2 + Psi.Im_r[i] * Psi.Im_f[i]
    def set_potential(safe, V):
        for i in range(0, len(Psi.V)):
            Psi.V[i] = V[i] 
    def compute_energy(safe):
        A = hbar**2 / (2 * m * dx**2)
        ke = 0 + 0.j
        pe = 0 + 0.j
        psi = []
        for i in range(0, len(Psi.Re)):
            psi.append(Psi.Re[i] + Psi.Im[i] * 1.j)
            pe += psi[i] * psi[i].conjugate() * Psi.V[i]
        for i in range(1, len(Psi.Re) - 1):
            Lap = psi[i + 1] - 2 * psi[i] + psi[i - 1]
            ke += Lap * psi[i].conjugate() * (Psi.X[i] - Psi.X[i - 1])
        KE = -A * np.real(ke)/(1.6 * 10**(-19))
        PE = np.real(pe)
        return (PE + KE) 

In [29]:
psi = Psi()
psi.initialise()
psi.compute_energy() # w eV

0.03805437270042673

In [30]:
fig = plt.figure(figsize = (12,6))
line1, = plt.plot([], [],'C0')
line2, = plt.plot([], [],'C1')
line3, = plt.plot([], [],'C2')
plt.grid(linestyle = '--',linewidth = 1)
plt.ylim(-0.5,1.5)
plt.xlim(-10, 10)
plt.xlabel('$x$')
plt.ylabel('$\Psi(x,t)$')

plt.close()

   
def frame(i):
    line1.set_data(psi.X, psi.Re)
    line2.set_data(psi.X, psi.Im)
    line3.set_data(psi.X, psi.P)
    if(i > 0):
        psi.update(20)
    
    return line1, line2, line3



animation = FuncAnimation(fig, frame, frames = 50, interval = 20)

#file = r"swobodna_stacjonarna.html"
#writervideo = anim.HTMLWriter(fps=100) 
#animation.save(file, writer=writervideo)

HTML(animation.to_jshtml())


# Biegnąca cząstka swobodna: $V(x) = 0$ 

In [7]:
dt = 0.001 * 10**(-15)

In [15]:
class moving_Psi:
    ra = (hbar/(2*m)) * (dt/(dx**2))
    a = 1.0
    b = 5.0
    Re = []
    Im = []
    Im_r = []
    Im_f = []
    P = []
    X = []
    V = []
    def initialise(safe):
        x = -15
        while(x <= 15):
            moving_Psi.Re.append(gauss((x + 5), moving_Psi.a) * np.cos(moving_Psi.b * (x + 5)))
            moving_Psi.Im.append(gauss((x + 5), moving_Psi.a) * np.sin(moving_Psi.b * (x + 5)))
            moving_Psi.Im_r.append(0.0)
            moving_Psi.Im_f.append(0.0)
            moving_Psi.V.append(0.0)
            moving_Psi.P.append((gauss((x + 5), moving_Psi.a) * np.cos(moving_Psi.b * (x + 5)))**2 + (gauss((x + 5), moving_Psi.a) * np.sin(moving_Psi.b * (x + 5)))**2)
            moving_Psi.X.append(x)
            x += 0.04 
    def update(safe, n):
        Re = moving_Psi.Re
        Im = moving_Psi.Im
        for k in range(0, n):
            for i in range(1, len(moving_Psi.Re) - 1):
                moving_Psi.Re[i] = Re[i] - moving_Psi.ra * (Im[i + 1] - 2 * Im[i] + Im[i - 1]) + dt/hbar * moving_Psi.V[i] * Im[i]
                moving_Psi.Im[i] = Im[i] + moving_Psi.ra * (Re[i + 1] - 2 * Re[i] + Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.Im_r[i] = Im[i]
                moving_Psi.Im_f[i] = moving_Psi.Im[i] + moving_Psi.ra * (moving_Psi.Re[i + 1] - 2 * moving_Psi.Re[i] + moving_Psi.Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.P[i] = moving_Psi.Re[i]**2 + moving_Psi.Im_r[i] * moving_Psi.Im_f[i]
    def set_potential(safe, V):
        for i in range(0, len(moving_Psi.V)):
            moving_Psi.V[i] = V[i]
    def compute_energy(safe):
        A = hbar**2 / (2 * m * dx**2)
        ke = 0 + 0.j
        pe = 0 + 0.j
        psi = []
        for i in range(0, len(moving_Psi.Re)):
            psi.append(moving_Psi.Re[i] + moving_Psi.Im[i] * 1.j)
            pe += psi[i] * psi[i].conjugate() * moving_Psi.V[i]
        for i in range(1, len(moving_Psi.Re) - 1):
            Lap = psi[i + 1] - 2 * psi[i] + psi[i - 1]
            ke += Lap * psi[i].conjugate() * (moving_Psi.X[i] - moving_Psi.X[i - 1])
        KE = -A * np.real(ke)/(1.6 * 10**(-19))
        PE = np.real(pe)
        return (PE + KE)        

In [16]:
psi = moving_Psi()
psi.initialise()
psi.compute_energy()

0.1580702377115033

In [26]:
fig = plt.figure(figsize = (12,6))
line1, = plt.plot([], [],'C0')
line2, = plt.plot([], [],'C1')
line3, = plt.plot([], [],'C2')
plt.grid(linestyle = '--',linewidth = 1)
plt.ylim(-1,1)
plt.xlim(-10, 10)
plt.xlabel('$x$')
plt.ylabel('$\Psi(x,t)$')

plt.close()

   
def frame(i):
    line1.set_data(psi.X, psi.Re)
    line2.set_data(psi.X, psi.Im)
    line3.set_data(psi.X, psi.P)
    psi.update(350)
    
    return line1, line2, line3



animation = FuncAnimation(fig, frame, frames = 1, interval = 20)

#file = r"swobodna_biegnaca.html"
#writervideo = anim.HTMLWriter(fps=100) 
#animation.save(file, writer=writervideo)

HTML(animation.to_jshtml())

# Schodek potenjału: $V(x) = \left\{ \begin{array}{cc}  
0.0 \text{ eV} & \text{ dla } x < 0 \\
0.1 \text{ eV} & \text{ dla } x > 0 \\
\end{array} \right.$ 

In [20]:
class moving_Psi:
    ra = (hbar/(2*m)) * (dt/(dx**2))
    a = 1.0
    b = 5.0
    Re = []
    Im = []
    Im_r = []
    Im_f = []
    P = []
    X = []
    V = []
    def initialise(safe):
        x = -20
        while(x <= 20):
            moving_Psi.Re.append(gauss((x + 5), moving_Psi.a) * np.cos(moving_Psi.b * (x + 5)))
            moving_Psi.Im.append(gauss((x + 5), moving_Psi.a) * np.sin(moving_Psi.b * (x + 5)))
            moving_Psi.Im_r.append(0.0)
            moving_Psi.Im_f.append(0.0)
            moving_Psi.V.append(0.0)
            moving_Psi.P.append((gauss((x + 5), moving_Psi.a) * np.cos(moving_Psi.b * (x + 5)))**2 + (gauss((x + 5), moving_Psi.a) * np.sin(moving_Psi.b * (x + 5)))**2)
            moving_Psi.X.append(x)
            x += 0.04 
    def update(safe, n):
        Re = moving_Psi.Re
        Im = moving_Psi.Im
        for k in range(0, n):
            for i in range(1, len(moving_Psi.Re) - 1):
                moving_Psi.Re[i] = Re[i] - moving_Psi.ra * (Im[i + 1] - 2 * Im[i] + Im[i - 1]) + dt/hbar * moving_Psi.V[i] * Im[i]
                moving_Psi.Im[i] = Im[i] + moving_Psi.ra * (Re[i + 1] - 2 * Re[i] + Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.Im_r[i] = Im[i]
                moving_Psi.Im_f[i] = moving_Psi.Im[i] + moving_Psi.ra * (moving_Psi.Re[i + 1] - 2 * moving_Psi.Re[i] + moving_Psi.Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.P[i] = moving_Psi.Re[i]**2 + moving_Psi.Im_r[i] * moving_Psi.Im_f[i]
    def set_potential(safe, V):
        for i in range(0, len(moving_Psi.V)):
            moving_Psi.V[i] = V[i] 
    def compute_energy(safe):
        A = hbar**2 / (2 * m * dx**2)
        ke = 0 + 0.j
        pe = 0 + 0.j
        psi = []
        for i in range(0, len(moving_Psi.Re)):
            psi.append(moving_Psi.Re[i] + moving_Psi.Im[i] * 1.j)
            pe += psi[i] * psi[i].conjugate() * moving_Psi.V[i]
        for i in range(1, len(moving_Psi.Re) - 1):
            Lap = psi[i + 1] - 2 * psi[i] + psi[i - 1]
            ke += Lap * psi[i].conjugate() * (moving_Psi.X[i] - moving_Psi.X[i - 1])
        KE = -A * np.real(ke)/(1.6 * 10**(-19))
        PE = np.real(pe)
        return (PE + KE)           
            

In [21]:
# Definicja potencjału
V = []
v = []
X = []
x = -20
while(x <= 20):
    if(x <= 0):
        V.append(0)
        v.append(0)
        X.append(x)
        if(x > -0.04):
            X.append(x)
    else:
        V.append(0.1 * 1.6 * 10**(-19))
        v.append(0.5)
        if(x < 20):            
            X.append(x)
    x += 0.04 
v.append(0.5)    

In [22]:
dt = 0.001 * 10**(-15)
psi = moving_Psi()
psi.initialise()
psi.set_potential(V)
psi.compute_energy()

0.15807023771150325

In [150]:
fig = plt.figure(figsize = (16,6))
line1, = plt.plot([], [],'C0')
line2, = plt.plot([], [],'C1')
line3, = plt.plot([], [],'C2')
line4, = plt.plot([], [],'C5', linestyle = '--', linewidth = 1)
plt.ylim(-1,1)
plt.xlim(-15, 15)
plt.xlabel('$x$')
plt.ylabel('$\Psi(x,t)$')

plt.close()

   
def frame(i):
    line4.set_data(X, v)
    line1.set_data(psi.X, psi.Re)
    line2.set_data(psi.X, psi.Im)
    line3.set_data(psi.X, psi.P)    
    psi.update(400)
    
    return line1, line2, line3, line4


animation = FuncAnimation(fig, frame, frames = 400, interval = 20)

#file = r"schodek.html"
#writervideo = anim.HTMLWriter(fps=100) 
#animation.save(file, writer=writervideo)

HTML(animation.to_jshtml())

# Bariera potenjału: $V(x) = \left\{ \begin{array}{cc}  
0.3\text{ eV} & \text{ dla } x \in \left(-0.5, 0.5\right) \\
0.0\text{ eV} & \text{w p.p. } \\
\end{array} \right.$ 

In [23]:
dt = 0.001 * 10**(-15)

In [24]:
class moving_Psi:
    ra = (hbar/(2*m)) * (dt/(dx**2))
    a = 1.0
    b = 5.0
    Re = []
    Im = []
    Im_r = []
    Im_f = []
    P = []
    X = []
    V = []
    def initialise(safe):
        x = -20
        while(x <= 20):
            moving_Psi.Re.append(gauss((x + 5), moving_Psi.a) * np.cos(moving_Psi.b * (x + 5)))
            moving_Psi.Im.append(gauss((x + 5), moving_Psi.a) * np.sin(moving_Psi.b * (x + 5)))
            moving_Psi.Im_r.append(0.0)
            moving_Psi.Im_f.append(0.0)
            moving_Psi.V.append(0.0)
            moving_Psi.P.append((gauss((x + 5), moving_Psi.a) * np.cos(moving_Psi.b * (x + 5)))**2 + (gauss((x + 5), moving_Psi.a) * np.sin(moving_Psi.b * (x + 5)))**2)
            moving_Psi.X.append(x)
            x += 0.04 
    def update(safe, n):
        Re = moving_Psi.Re
        Im = moving_Psi.Im
        for k in range(0, n):
            for i in range(1, len(moving_Psi.Re) - 1):
                moving_Psi.Re[i] = Re[i] - moving_Psi.ra * (Im[i + 1] - 2 * Im[i] + Im[i - 1]) + dt/hbar * moving_Psi.V[i] * Im[i]
                moving_Psi.Im[i] = Im[i] + moving_Psi.ra * (Re[i + 1] - 2 * Re[i] + Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.Im_r[i] = Im[i]
                moving_Psi.Im_f[i] = moving_Psi.Im[i] + moving_Psi.ra * (moving_Psi.Re[i + 1] - 2 * moving_Psi.Re[i] + moving_Psi.Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.P[i] = moving_Psi.Re[i]**2 + moving_Psi.Im_r[i] * moving_Psi.Im_f[i]
    def set_potential(safe, V):
        for i in range(0, len(moving_Psi.V)):
            moving_Psi.V[i] = V[i] 
    def compute_energy(safe):
        A = hbar**2 / (2 * m * dx**2)
        ke = 0 + 0.j
        pe = 0 + 0.j
        psi = []
        for i in range(0, len(moving_Psi.Re)):
            psi.append(moving_Psi.Re[i] + moving_Psi.Im[i] * 1.j)
            pe += psi[i] * psi[i].conjugate() * moving_Psi.V[i]
        for i in range(1, len(moving_Psi.Re) - 1):
            Lap = psi[i + 1] - 2 * psi[i] + psi[i - 1]
            ke += Lap * psi[i].conjugate() * (moving_Psi.X[i] - moving_Psi.X[i - 1])
        KE = -A * np.real(ke)/(1.6 * 10**(-19))
        PE = np.real(pe)
        return (PE + KE) 
        
    

In [25]:
# Definicja potencjału
V = []
v = []
X = []
x = -20
while(x <= 20):
    if(x >= -0.5 and x <= 0.5):
        V.append(0.3 * 1.6 * 10**(-19))
        X.append(x)
        v.append(0.5)
        if(x < -0.47 or x > 0.47):
            X.append(x)
            v.append(0.0)
                
    else:
        V.append(0.0)
        X.append(x)
        v.append(0.0)
    x += 0.04
    
psi = moving_Psi()
psi.initialise()
psi.set_potential(V) 
psi.compute_energy()

0.15807023771150325

In [45]:
fig = plt.figure(figsize = (16,8))
line4, = plt.plot([], [],'C4')
line1, = plt.plot([], [],'C0')
line2, = plt.plot([], [],'C1')
line3, = plt.plot([], [],'C2')

plt.grid(linestyle = '--',linewidth = 1)
plt.ylim(-1, 1)
plt.xlim(-15, 15)
plt.xlabel('$x$')
plt.ylabel('$\Psi(x,t)$')

plt.close()

def frame(i):
    line4.set_data(X, v)
    line1.set_data(psi.X, psi.Re)
    line2.set_data(psi.X, psi.Im)
    line3.set_data(psi.X, psi.P)    
    if(i > 0):
        psi.update(400)
    
    return line1, line2, line3



animation = FuncAnimation(fig, frame, frames = 400, interval = 20)

#file = r"bariera_IV.html"
#writervideo = anim.HTMLWriter(fps=100) 
#animation.save(file, writer=writervideo)

HTML(animation.to_jshtml())